In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from itertools import combinations

import gurobipy as gp
import numpy as np
from gurobipy import GRB
from scipy.spatial.distance import squareform

<IPython.core.display.Javascript object>

## Helper functions

In [3]:
def subtourelim(model, where):
    """
    Callback function. Use lazy constraints to eliminate subtours.
    """
    if where == GRB.Callback.MIPSOL:

        # Make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist(
            (i, j) for i, j in model._vars.keys() if vals[i, j] > 0.5
        )

        # Find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
            # Add subtour elimination constraints for every pair of cities in tour
            model.cbLazy(
                gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                <= len(tour) - 1
            )

<IPython.core.display.Javascript object>

In [4]:
def subtour(edges):
    """
    Given a tuplelist of edges, find the shortest subtour
    """
    unvisited = list(range(n))
    cycle = range(n + 1)  # initial length has 1 more city
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, "*") if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

<IPython.core.display.Javascript object>

## Load the data matrix

In [5]:
with open("usa50.txt") as f:
    dists = f.read()

    # Convert into 1D array
    arr = np.fromstring(dists, sep=" ")
    arr = arr[np.nonzero(arr)]

    # Convert to symmetric matrix
    mat = squareform(arr)

<IPython.core.display.Javascript object>

In [6]:
# Make a dictionary of Euclidean distance between each pair of points
n = mat.shape[0]
dist = {(i, j): mat[i, j] for i in range(n) for j in range(i)}

<IPython.core.display.Javascript object>

## Build the model

In [7]:
# Initialize the model
m = gp.Model()

# Create variables
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name="e")
for i, j in vars.keys():
    vars[j, i] = vars[i, j]

# Add degree-2 constraint
m.addConstrs(vars.sum(i, "*") == 2 for i in range(n))

Using license file /Users/vivek/gurobi.lic
Academic license - for non-commercial use only


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

<IPython.core.display.Javascript object>

In [8]:
%%time
# Optimize model
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 50 rows, 1225 columns and 2450 nonzeros
Model fingerprint: 0x600a73af
Variable types: 0 continuous, 1225 integer (1225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+04, 6e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective 1.108853e+08
Presolve time: 0.00s
Presolved: 50 rows, 1225 columns, 2450 nonzeros
Variable types: 0 continuous, 1225 integer (1225 binary)

Root relaxation: objective 1.870668e+07, 76 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.8707e+07    0   14 1.1089e+08 1.8707e+07  83.1%     -    0s
H    0     0                    1.967024e+07 1.8707e+07  4.90%     -    0s
H 

<IPython.core.display.Javascript object>

In [9]:
# Unpack the results
vals = m.getAttr("x", vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

print("")
print("Optimal tour: %s" % str(tour))
print("Optimal cost: %g" % m.objVal)
print("")


Optimal tour: [0, 10, 37, 20, 6, 19, 9, 18, 22, 11, 12, 4, 23, 46, 47, 21, 25, 31, 49, 32, 13, 29, 24, 5, 26, 16, 48, 40, 27, 30, 17, 44, 28, 33, 34, 45, 14, 15, 41, 8, 43, 1, 7, 38, 3, 39, 2, 42, 35, 36]
Optimal cost: 1.87607e+07



<IPython.core.display.Javascript object>